In [1]:
from itertools import product
import pickle

import numpy as np

from algorithms.svr import SupportVectorRegressor
from algorithms.preprocessing import load_and_preprocess_dataset

import config

In [2]:
svr_logs_dir = config.LOGS_DIR / 'svr'
svr_logs_dir.mkdir(exist_ok=True)

In [3]:
X, y, X_test, y_test = load_and_preprocess_dataset()

In [4]:
X = np.asarray(X)
y = np.asarray(y)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [12]:
hyperparameters = [
    #(0.1, 0.001, 0.01, 'linear', 0.5),
    #(0.1, 0.001, 0.01, 'rbf', 0.5),
    #(0.1, 0.001, 0.001, 'linear', 0.5),
    #(0.1, 0.001, 0.001, 'rbf', 0.5),
    #(0.1, 0.001, 0.001, 'rbf', 1),
    (0.1, 0.01, 0.01, 'linear', 0.5),
    (0.1, 0.01, 0.001, 'rbf', 0.5),
    (0.2, 0.001, 0.001, 'rbf', 0.5),
    (0.2, 0.001, 0.001, 'rbf', 1),
    (0.2, 0.01, 0.001, 'linear', 1),
    (0.2, 0.01, 0.001, 'rbf', 1),
]

In [ ]:
indices = np.random.permutation(len(y))
fold_size = len(indices) // 5
scores = []

for C, epsilon, tolerance, kernel_type, gamma in hyperparameters:
    fold_scores = []
    for fold in range(5):
        valid_indices = indices[fold * fold_size: (fold + 1) * fold_size]
        train_indices = indices[~np.isin(indices, valid_indices)]

        X_train, y_train = X[train_indices], y[train_indices]
        X_valid, y_valid = X[valid_indices], y[valid_indices]

        model = SupportVectorRegressor(X_train, y_train, C, epsilon, tolerance, kernel_type=kernel_type, gamma=gamma)
        history = model.fit(X_valid, y_valid, max_iterations=250)
        fold_scores.append(history)

    with open(svr_logs_dir / f'C_{C}-epsilon_{epsilon}-tolerance_{tolerance}-kernel_type_{kernel_type}-gamma_{gamma}.pkl', 'wb') as f:
        pickle.dump(fold_scores, f)

C=0.2, epsilon=0.001, tolerance=0.001:  18%|█▊        | 44/250 [03:05<13:18,  3.88s/it, train_mse=0.2086690, valid_mse=0.2421145, number of changes: 1714]